In [8]:
import numpy as np
import pandas as pd
from PIL import Image, ImageEnhance

In [2]:
from google.cloud import vision
import io

In [23]:
def detect_text(path):
    """Detects text in the file."""
    
    client = vision.ImageAnnotatorClient()

#     img = PIL.Image.open(path)
#     enhancer = ImageEnhance.Contrast(img)
    
#     img2 = enhancer.enhance(1.5)
#     img2.save(path)
    
    with io.open(path, 'rb') as image_file:
        content = image_file.read()
    
    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
#     print(texts)
    print('Texts:')

    for text in texts:
        print('\n"{}"'.format(text.description))

        vertices = (['({},{})'.format(vertex.x, vertex.y)
                    for vertex in text.bounding_poly.vertices])

        print('bounds: {}'.format(','.join(vertices)))

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))
    return texts

In [24]:
texts = detect_text('./bill2.jpeg')

Texts:

"HSN CODE Item Descr iption Net Price Oty Value
3304 EAN: 8901030772290
PONDS DREAM FLONER TALC 400g PET
230 .00 1 230.00
19059040 EAN: 8901441017010
64.80
64.80
LIJJAT PUNJABI MSL PAPAD SPL 200 g PP
165.00
1 165.00
3304 EAN: 8901030642784
PONDS HHITE BEAUTY ANTY CREAM 50 G CBD
190590 EAN: 8901063023901
27.00 2 54.00
BRITANNIA MARIEGOLD BISCUIT 250 G PP
1902
EAN: 8904132912854
35.00 1
35.00
SNACTAC MASALA NOODLES 300 g pp
"
bounds: (29,33),(655,33),(655,442),(29,442)

"HSN"
bounds: (36,38),(65,38),(65,58),(36,58)

"CODE"
bounds: (77,34),(118,34),(118,55),(77,55)

"Item"
bounds: (134,33),(174,33),(174,52),(134,52)

"Descr"
bounds: (188,33),(240,33),(240,51),(188,51)

"iption"
bounds: (248,33),(308,33),(308,51),(248,51)

"Net"
bounds: (367,33),(397,33),(397,51),(367,51)

"Price"
bounds: (412,33),(464,33),(464,50),(412,50)

"Oty"
bounds: (501,33),(530,33),(530,50),(501,50)

"Value"
bounds: (588,35),(639,35),(639,51),(588,51)

"3304"
bounds: (37,108),(76,107),(76,126),(37,127)

"EA

In [27]:
text = texts[0].description

In [30]:
print(text.split('\t'))

['HSN CODE Item Descr iption Net Price Oty Value\n3304 EAN: 8901030772290\nPONDS DREAM FLONER TALC 400g PET\n230 .00 1 230.00\n19059040 EAN: 8901441017010\n64.80\n64.80\nLIJJAT PUNJABI MSL PAPAD SPL 200 g PP\n165.00\n1 165.00\n3304 EAN: 8901030642784\nPONDS HHITE BEAUTY ANTY CREAM 50 G CBD\n190590 EAN: 8901063023901\n27.00 2 54.00\nBRITANNIA MARIEGOLD BISCUIT 250 G PP\n1902\nEAN: 8904132912854\n35.00 1\n35.00\nSNACTAC MASALA NOODLES 300 g pp\n']
